In [6]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [7]:
!kaggle competitions download -c dogs-vs-cats

100% 810M/812M [00:38<00:00, 22.8MB/s]
100% 812M/812M [00:38<00:00, 22.0MB/s]


In [8]:
!unzip -qq dogs-vs-cats.zip

In [9]:
!unzip -qq train.zip

In [10]:
import os, shutil, pathlib

original_dir = pathlib.Path("train")
new_base_dir = pathlib.Path("cats_vs_dogs_small")

def make_subset(subset_name, start_index, end_index):
    for category in ("cat", "dog"):
        dir = new_base_dir / subset_name / category
        os.makedirs(dir, exist_ok=True)
        fnames = [f"{category}.{i}.jpg" for i in range(start_index, end_index)]
        for fname in fnames:
            shutil.copyfile(src=original_dir / fname,
                            dst=dir / fname)

make_subset("train", start_index=0, end_index=1000)
make_subset("validation", start_index=1000, end_index=1500)
make_subset("test", start_index=1500, end_index=2500)

In [11]:
from tensorflow import keras
from keras.utils import image_dataset_from_directory

train_dataset = image_dataset_from_directory("/content/cats_vs_dogs_small/train", image_size=(180,180), batch_size=32)
validation_dataset = image_dataset_from_directory("/content/cats_vs_dogs_small/validation", image_size=(180,180), batch_size=32)
test_dataset = image_dataset_from_directory("/content/cats_vs_dogs_small/test", image_size=(180,180), batch_size=32)

Found 2000 files belonging to 2 classes.
Found 1000 files belonging to 2 classes.
Found 2000 files belonging to 2 classes.


In [12]:
conv_base = keras.applications.vgg16.VGG16(
    weights='imagenet',
    include_top=False,
    input_shape=(180,180,3)
)

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


In [13]:
for layer in conv_base.layers[:-4]:
  layer.trainable = False

In [14]:
from keras import layers

inputs = keras.Input(shape=(180,180,3))

data_augmentation = keras.Sequential([
    layers.RandomFlip('horizontal'),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.2)
])

x = data_augmentation(inputs)
x = keras.applications.vgg16.preprocess_input(x)
x = conv_base(x)
x = layers.Flatten()(x)
x = layers.Dense(216, activation='relu')(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model = keras.Model(inputs, outputs)

model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.RMSprop(learning_rate=1e-5), metrics=['accuracy'])

callbacks = [keras.callbacks.ModelCheckpoint(filepath='finetuningpretrainedmodel.keras', save_best_only=True, monitor='val_loss')]

history = model.fit(train_dataset, epochs=30, validation_data=validation_dataset, callbacks=callbacks)

Epoch 1/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 25s 254ms/step - accuracy: 0.6254 - loss: 4.6359 - val_accuracy: 0.8970 - val_loss: 0.5938
Epoch 2/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 14s 227ms/step - accuracy: 0.8119 - loss: 1.2677 - val_accuracy: 0.9380 - val_loss: 0.2884
Epoch 3/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 19s 199ms/step - accuracy: 0.8475 - loss: 0.7730 - val_accuracy: 0.9450 - val_loss: 0.2031
Epoch 4/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 20s 200ms/step - accuracy: 0.8714 - loss: 0.4413 - val_accuracy: 0.9560 - val_loss: 0.1639
Epoch 5/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 21s 202ms/step - accuracy: 0.9113 - loss: 0.2982 - val_accuracy: 0.9530 - val_loss: 0.1522
Epoch 6/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 15s 232ms/step - accuracy: 0.9274 - loss: 0.2434 - val_accuracy: 0.9590 - val_loss: 0.1292
Epoch 7/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 12s 193ms/step - accuracy: 0.9466 - loss: 0.1712 - val_accuracy: 0.9590 - val_loss: 0.1364
Epoch 8/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 21s 207ms/step - accuracy: 0.9508 - loss: 0.1316 - val_accu

In [15]:
model.evaluate(test_dataset)

63/63 ━━━━━━━━━━━━━━━━━━━━ 7s 108ms/step - accuracy: 0.9726 - loss: 0.1841


[0.1646195501089096, 0.9745000004768372]